<a href="https://colab.research.google.com/github/clionelove123/Machine_Learning/blob/main/ML_Chap_10_Genetic_Algorithm_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [181]:
# 코드 출처: https://github.com/Junst/TSP-Genetic-Algorithm/tree/master

import random
import math
import numpy as np
import csv
import matplotlib.pyplot as plt
import time

In [182]:
MUTATION_RATE = 40
MUTATION_COUNT = 2
THRESHOLD = 35000
UNIFORMCROSSOVER_RATE = 0.5

csvfile = '/content/TSP.csv'

In [183]:
def read_csv(csvf): #csv 파일 읽기, csv file read
    City = np.genfromtxt(open(csvf, "rb"),dtype=float, delimiter=",", skip_header=0)
    print(City)
    return City

cityCoordinates = read_csv(csvfile)
citySize = len(read_csv(csvfile))


[[53.25009344 22.44488521]
 [21.57092818 17.44707561]
 [72.16107676 85.88179689]
 ...
 [ 6.28095388 87.71359465]
 [81.30504684 27.76985517]
 [45.90940209 22.3171196 ]]
[[53.25009344 22.44488521]
 [21.57092818 17.44707561]
 [72.16107676 85.88179689]
 ...
 [ 6.28095388 87.71359465]
 [81.30504684 27.76985517]
 [45.90940209 22.3171196 ]]


In [184]:
class Genome():
    chromosomes = []
    fitness = 100000

    def __init__(self, numberOfchromosomes=None):
        if numberOfchromosomes is not None:
            self.chromosomes = list(range(numberOfchromosomes))
            randShuffle(self.chromosomes)


In [185]:
def randShuffle(listToShuffle):
    return random.shuffle(listToShuffle)


In [186]:
def init_population(size) :
    initial_population = []
    for i in range(size):
        newGenome = Genome()
        newGenome.chromosomes = random.sample(range(1, citySize), citySize - 1)
        newGenome.chromosomes.insert(0, 0)
        newGenome.chromosomes.append(0)
        newGenome.fitness = Evaluate(newGenome.chromosomes)
        initial_population.append(newGenome)
    return initial_population

In [187]:
def Evaluate(chromosomes):
    Fitness = 0
    for i in range(len(chromosomes) - 1):
        p1 = cityCoordinates[chromosomes[i]]
        p2 = cityCoordinates[chromosomes[i + 1]]
        Fitness += Euclidean_distance(p1, p2)
    Fitness = np.round(Fitness, 2)
    return Fitness

In [188]:
def Euclidean_distance(x, y):
    dist = np.linalg.norm(np.array(x)-np.array(y))
    return dist

In [189]:
def findBestGenome(population):
    allFitness = [i.fitness for i in population]
    bestFitness = min(allFitness)
    return population[allFitness.index(bestFitness)]

In [190]:
def TournamentSelection(population, k):
    select = [population[random.randrange(0, len(population))] for i in range(k)]
    bestGenome = findBestGenome(select)
    return bestGenome

In [191]:
def Reproduction(population):
    parent1 = TournamentSelection(population, 15).chromosomes
    parent2 = TournamentSelection(population, 15).chromosomes
    while parent1 == parent2:
        parent2 = TournamentSelection(population, 15).chromosomes

    return OrderCrossover(parent1, parent2)

def randRange(first,last):
    return random.randint(first,last)


In [192]:
def OrderCrossover(parent1, parent2):
    child = Genome(None)
    child.chromosomes = []
    firstIndex = randRange(0,len(parent1)-1)
    secondIndex = randRange(firstIndex, len(parent1)-1)
    innerSet = parent1[firstIndex:secondIndex]
    startSet = []
    endSet = []
    for _, value in enumerate([item for item in parent2 if item not in innerSet]):
        if len(startSet)<firstIndex:
            startSet.append(value)
        else:
            endSet.append(value)
    child.chromosomes = startSet + innerSet + endSet

    if random.randrange(0, 100) < MUTATION_RATE:
        child.chromosomes = InversionMutation(child.chromosomes)

    child.fitness = Evaluate(child.chromosomes)
    return child


In [193]:
def InversionMutation(chromo) :
    for x in range(MUTATION_COUNT):
        p1, p2 = [random.randrange(1, len(chromo) - 1) for i in range(2)]
        while p1 == p2 or p1 > p2:
            p1 = random.randint(0, len(chromo) - 1)
            p2 = random.randint(0, len(chromo) - 1)
        log = chromo[p1:p2]
        log = log[::-1]
        chromo = chromo[:p1] + log +chromo[p2:]
    return chromo

In [194]:
def fitness_plot(generation, allBestFitness):
    plt.plot(range(0, generation), allBestFitness, c='blue')
    plt.xlabel('Generations')
    plt.ylabel('Best Fitness')
    plt.title('Fitness Function')
    plt.show()

In [195]:
def city_visualize(bestGenome, city):
    start = None
    for x, y in city:
        if start is None: #시작지점이면 표시
            start = city[0]
            plt.scatter(start[0], start[1], c="green", marker=">")
            plt.annotate("Start", (x + 2, y - 2), color='red')
        else: #시작지점 아니면
            plt.scatter(x, y, marker='.', s=10, c="black")

    #edge 표현을 위한 x, y 범위
    x_edge = [city[i][0] for i in bestGenome.chromosomes]
    y_edge = [city[i][1] for i in bestGenome.chromosomes]

    plt.plot(x_edge, y_edge, color="blue", linewidth=0.07, linestyle="-")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('City Edges')
    plt.show()

In [196]:
def GeneticAlgorithm(populationSize, Generation_Count):
    allBestFitness = []
    population = init_population(populationSize)
    generation = 0
    TotalBestFitness = 100000
    TotalBestPath = []

    start = time.time()

    while generation < Generation_Count:
        generation += 1

        for i in range(populationSize):
            population.append(Reproduction(population))

        for genom in population:
            if genom.fitness > THRESHOLD:
                population.remove(genom)

        averageFitness = round(np.sum([genom.fitness for genom in population]) / len(population), 2)
        bestGenome = findBestGenome(population)
        end = time.time()

        if bestGenome.fitness < TotalBestFitness:
            TotalBestFitness = bestGenome.fitness
            TotalBestPath = bestGenome.chromosomes
            print("\n" * 5)
            print("Generation: {0}\nPopulation Size: {1}\t Average Fitness: {2}\nBest Fitness: {3}".format(generation, len(population), averageFitness,bestGenome.fitness))
            print("Total Best Fitness : ",TotalBestFitness)
            print("Total time : ", end-start) # 소요 시간 표기, Working Time
            print("Best Path:",TotalBestPath)
            allBestFitness.append(bestGenome.fitness)

    return bestGenome.chromosomes

In [197]:
solution=GeneticAlgorithm(populationSize=30, Generation_Count=5000) #Population size, Generation Count 입력







Generation: 1
Population Size: 30	 Average Fitness: 51834.69
Best Fitness: 50253.41
Total Best Fitness :  50253.41
Total time :  0.23791241645812988
Best Path: [0, 793, 791, 650, 943, 921, 335, 802, 745, 776, 429, 1, 819, 617, 740, 394, 807, 689, 583, 847, 152, 794, 917, 891, 622, 559, 685, 773, 477, 640, 950, 785, 402, 481, 732, 77, 876, 638, 941, 479, 337, 861, 247, 486, 751, 744, 424, 307, 571, 995, 146, 705, 367, 648, 371, 294, 715, 212, 96, 198, 68, 882, 526, 730, 906, 552, 556, 207, 50, 783, 974, 9, 923, 815, 888, 472, 534, 400, 66, 902, 546, 673, 824, 267, 438, 561, 180, 18, 63, 321, 343, 764, 542, 753, 591, 574, 269, 884, 317, 265, 245, 476, 582, 379, 609, 823, 672, 772, 387, 73, 834, 158, 484, 528, 244, 110, 595, 439, 352, 688, 644, 946, 924, 302, 38, 621, 821, 225, 833, 470, 390, 453, 93, 683, 714, 25, 503, 175, 58, 786, 17, 723, 537, 60, 312, 119, 444, 900, 842, 64, 938, 548, 830, 228, 254, 618, 841, 898, 735, 859, 107, 814, 604, 10, 67, 160, 679, 127, 904, 948, 241, 8